In [1]:
from tqdm import tqdm
from glob import glob
import os
from PIL import Image
from multiprocessing import Pool
from PIL import Image

In [2]:
color_temp_to_rgb = {}

with open("temp-to-rgb.txt") as f:
  for line in f:
    if line[0] == "#":
      continue
    
    line_respaced = ""
    for i in range(1, len(line)):
      if not (i > 0 and line[i] == ' ' and line[i-1] == ' '):
        line_respaced += line[i]
    
    line_respaced_split = line_respaced.split(" ")
    color_temp = int(line_respaced_split[0])
    r, g, b = [float(x) * 255 for x in line_respaced_split[6:9]]
    
    # 10deg figures overwrite the 2deg ones
    # so the actual RGB values are for CIE 1964 10 degree CMFs, whatever that means
    color_temp_to_rgb[color_temp] = (r, g, b)

In [3]:
def convert_img_to_temp(img, temp):
  r, g, b = color_temp_to_rgb[temp]
  matrix = (r / 255., 0., 0., 0.,
            0., g / 255., 0., 0.,
            0., 0., b / 255., 0.,)
  return img.convert("RGB", matrix)

def load_and_convert_img(img_path, output_dirname, temp):
    img = Image.open(img_path).convert("RGB")
    output_path = f"{output_dirname}/{img_path.split('/')[-1][:-4]}.png"
    convert_img_to_temp(img, temp).save(output_path)
    return output_path

In [4]:
temp = 5000

train_output_dirname = f"../RT-DETR/dataset/train-temp={temp}"
val_output_dirname = f"../RT-DETR/dataset/val-temp={temp}"
test_output_dirname = f"../RT-DETR/dataset/test-temp={temp}"
suspects_output_dirname = f"../RT-DETR/dataset/suspects-temp={temp}"

os.makedirs(train_output_dirname, exist_ok=True)
os.makedirs(val_output_dirname, exist_ok=True)
os.makedirs(test_output_dirname, exist_ok=True)
os.makedirs(suspects_output_dirname, exist_ok=True)

def shift_train(img_path):
    return load_and_convert_img(img_path, train_output_dirname, temp)

def shift_val(img_path):
    return load_and_convert_img(img_path, val_output_dirname, temp)

def shift_test(img_path):
    return load_and_convert_img(img_path, test_output_dirname, temp)

def shift_suspects(img_path):
    return load_and_convert_img(img_path, suspects_output_dirname, temp)

# print("Shifting train images...")
# with Pool(16) as pool:
#     output_paths = pool.map(shift_train, glob("../RT-DETR/dataset/train/images/*.png"))

# print("Shifting val images...")
# with Pool(16) as pool:
#     output_paths = pool.map(shift_val, glob("../RT-DETR/dataset/val/images/*.png"))

# print("Shifting test images...")
# with Pool(16) as pool:
#     output_paths = pool.map(shift_test, glob("../RT-DETR/dataset/test/images/*.png"))

print("Shifting suspect images...")
with Pool(16) as pool:
    output_paths = pool.map(shift_suspects, glob("../RT-DETR/dataset/reid/suspects/*.png"))

Shifting suspect images...
